## Festival Recommendation 

### Contents

- Imports and Options
- Formulas
- Read and Clean Scrapy Data
- Spotify API calls

#### Imports and Options

In [266]:
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pickle
import time
from sklearn import preprocessing
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import config

%matplotlib inline

pd.options.display.float_format = '{:20,.5f}'.format
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

#### Formulas

In [255]:
def date_split(dates, position=0):
    ''' Splits festival dates and returns start date(position = 0) or end date(position = 1)'''
    dates = dates.split('-')
    if position:
        end_date = dates[1]
        end_date = datetime.strptime(end_date, '%B %d, %Y')
        return end_date
    else:
        start_date = dates[0] + ' ' + dates[1][-4:]
        start_date = datetime.strptime(start_date, '%B %d %Y')
        return start_date
    

def make_lower(lineup):
    ''' Takes artist lineup and returns list with names lowcase and whitespace stripped'''
    lineup_new = []
    for artist in lineup:
        artist = artist.strip()
        artist = artist.lower()
        lineup_new.append(artist)
    return lineup_new
        

def genre_dict(lineup):
    '''Takes artist lineup and returns genre counts'''
    artist_genres = defaultdict(int)
    for artist in lineup:
        try:
            genres = artist_info[artist]['genres']
            for genre in genres:
                genre = genre.replace(' ', '_')
                genre = genre.replace('-', '_')
                artist_genres[genre] += 1
        except KeyError:
            next
    return artist_genres


def no_spotify_info(lineup):
    '''Takes artist lineup and returns artist not in Spotify database'''
    unknown = []
    for artist in lineup:
        try:
            genres = artist_info[artist]['genres']
        except KeyError:
            unknown.append(artist)
    return unknown


def user_genres(artists):
    genres = defaultdict(int)

    for artist in artists:
        try:
            genre_result = spotify.search(q='artist:' + artist, type='artist')['artists']['items'][0]['genres']
            for genre in genre_result:
                genre = genre.replace(' ', '_')
                genre = genre.replace('-', '_')
                genres[genre] += 1
        except IndexError:
            next
    df = pd.DataFrame(genres, index=["User",])
    df_norm = pd.DataFrame(preprocessing.normalize(df, norm='l1'), index=["User",], columns=list(df.columns))
    return df_norm

#### Read and Clean Scrapy Data

In [123]:
# Read in festival data
all_festivals = pd.read_json("pretty_festivals.json")

# Formatt columns
all_festivals['start_date'] = all_festivals.dates.apply(date_split)
all_festivals['end_date'] = all_festivals.dates.apply(lambda x: date_split(x, 1))
all_festivals['duration'] = all_festivals.end_date - all_festivals.start_date + timedelta(days=1)
all_festivals['lineup'] = all_festivals.lineup.apply(make_lower)


# Reorder columns and sort rows by date 
column_order = [
    'name', 
    'start_date', 
    'end_date', 
    'location', 
    'tickets', 
    'camping', 
    'website', 
    'description', 
    'lineup', 
    'url', 
    'poster', 
    'image'
]
all_festivals = all_festivals[column_order]
all_festivals.sort_values('start_date', inplace=True)
all_festivals = all_festivals.reset_index(drop=True)

# Remove festivals without lineups
have_lineup = []

for fest_lineup in all_festivals.lineup:
    if len(fest_lineup) >= 10:
        have_lineup.append(True)
    else:
        have_lineup.append(False)
        
festivals = all_festivals[have_lineup]
festivals = festivals.reset_index(drop=True)

# View dataframe head
festivals.head()

,name,start_date,end_date,location,tickets,camping,website,description,lineup,url,poster,image
0,Savannah Music Festival 2017,2017-03-23,2017-04-08,"Savannah, GA",,No,http://www.savannahmusicfestival.org/,THE SAVANNAH MUSIC FESTIVAL IS DEDICATED TO PR...,"[the avett brothers, jason isbell, nikki lane,...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...
1,Ultra Miami 2017,2017-03-24,2017-03-26,"Miami, FL",,No,http://www.ultramusicfestival.com/,"FOR EDM FANS AROUND THE WORLD, ULTRA IN MIAMI ...","[chase & status, cypress hill, ice cube, justi...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...
2,Winter Wonder Grass Tahoe 2017,2017-03-30,2017-04-02,"Squaw Valley, CA",,No,http://www.winterwondergrasstahoe.com/,"Lively national, regional and local bluegrass ...","[greensky bluegrass, yonder mountain string ba...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...
3,Desert Hearts 2017,2017-03-31,2017-04-03,"Warner Springs, CA",,Yes,http://www.deserthearts.us/,Desert Hearts is a bi-annual music and arts fe...,"[ardalan, atish, ben seagren, christian martin...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...
4,Fool's Paradise 2017,2017-03-31,2017-04-01,"St. Augustine, FL",$65-$250,Yes,http://www.foolsparadisefl.com/,Fool's Paradise is bringing your favorite band...,"[lettuce, dumpstaphunk, the floozies, joe russ...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...


#### Spotify API Calls (Spotify Data pickled below)

In [98]:
# Initialize Spotify API package
ccm = SpotifyClientCredentials(
    client_id = config.client_id, 
    client_secret = config.client_secret
)

spotify = spotipy.Spotify(client_credentials_manager=ccm)

In [ ]:
# Dictionary with artist info
artist_info = defaultdict(dict)

for lineup in all_festivals.lineup:
    for artist in lineup:
        search_result = spotify.search(q='artist:' + artist, type='artist')
        try:
            artist_info[artist] = search_result['artists']['items'][0]
        except IndexError:
            next
            
with open('artist_info.pk1', 'wb') as picklefile:
    pickle.dump(artist_info, picklefile)

In [ ]:
#artist = 'litz'
#search_result = spotify.search(q='artist:' + artist, type='artist')
#print search_result
#artist_info[artist] = search_result['artists']['items'][0]

In [15]:
with open('artist_tracks.pk1', 'rb') as picklefile:
    artist_tracks = pickle.load(picklefile)

In [12]:
# Dictionary with artist top tracks
artist_tracks = defaultdict(dict)

for artist, info in artist_info.items():
    try:
        artist_id = info['uri']
        top_tracks = spotify.artist_top_tracks(artist_id)
        artist_tracks[artist] = top_tracks
    except KeyError:
        next
        
with open('artist_tracks.pk1', 'wb') as picklefile:
    pickle.dump(artist_tracks, picklefile)

In [ ]:
# Dictionary with track features
audio_features = defaultdict(list)

for artist, track in tqdm(artist_tracks.items()):
    track_features = []
    for i in range(0, len(track['tracks'])):
        song_id = str(track['tracks'][i]['uri'])
        features = spotify.audio_features(song_id)
        track_features.append(features)
    audio_features[artist] = track_features

 86%|████████▌ | 3279/3808 [1:23:15<11:14,  1.28s/it]

In [ ]:
# pickel for later
with open('audio_features.pk1', 'wb') as picklefile:
    pickle.dump(audio_features, picklefile)

In [ ]:
with open('audio_features.pk1', 'rb') as picklefile:
    audio_features = pickle.load(picklefile)

In [24]:
artist_average = defaultdict(dict)

for artist, tracks in tqdm(audio_features.items()):
    averages = defaultdict(int)
    count = 0
    danceability = 0
    energy = 0
    key = 0
    loudness = 0
    speechiness = 0
    acousticness = 0
    instrumentalness = 0
    liveness = 0
    valence = 0
    tempo = 0
    for track in tracks:
        try:
            count += 1.0
            danceability += track[0]['danceability']
            energy += track[0]['energy']
            key += track[0]['key']
            loudness += track[0]['loudness']
            speechiness += track[0]['speechiness']
            acousticness += track[0]['acousticness']
            instrumentalness += track[0]['instrumentalness']
            liveness += track[0]['liveness']
            valence += track[0]['valence']
            tempo += track[0]['tempo']
        except TypeError:
            continue
    if count == 0:
        continue
    averages['danceability'] = danceability / count
    averages['energy'] = energy / count
    averages['key'] = key / count
    averages['loudness'] = loudness / count
    averages['speechiness'] = speechiness / count
    averages['acousticness'] = acousticness / count
    averages['instrumentalness'] = instrumentalness / count
    averages['liveness'] = liveness / count
    averages['valence'] = valence / count
    averages['tempo'] = tempo / count
    averages['genres'] = artist_info[artist]['genres']
    artist_average[artist] = averages

100%|██████████| 3808/3808 [00:00<00:00, 21084.11it/s]


In [27]:
# pickel for later
with open('artist_average.pk1', 'wb') as picklefile:
    pickle.dump(artist_average, picklefile)

### Music Festival Recommender

In [124]:
# load pickeled spotify data
with open('artist_info.pk1', 'rb') as picklefile:
    artist_info = pickle.load(picklefile)

In [125]:
# Add genre counts to dataframe
festivals['genres'] = festivals['lineup'].apply(genre_dict)

# List of bands not in spotify database
festivals['unknown'] = festivals['lineup'].apply(no_spotify_info)
festivals['num_bands'] = festivals['lineup'].apply(lambda x: len(x))
festivals['num_unknowns'] = festivals['unknown'].apply(lambda x: len(x))
festivals['unknown_percent'] = festivals['num_unknowns'] / festivals['num_bands']

In [128]:
# Make DataFrame of festival genres
first = festivals.ix[0,12]
fest_genres = pd.DataFrame(first, index=[festivals.ix[0,0],])

for i in range(1, len(festivals)):
    d = festivals.ix[i, 12]
    df = pd.DataFrame(d, index=[festivals.ix[i,0],])
    fest_genres = fest_genres.append(df)
    
fest_genres = fest_genres.fillna(0)

In [129]:
# Drop festivals with low percentage of bands in spotify database
skip = [
    "CMA Music Festival 2017", 
    "Mad Tea Party Jam 2017", 
    "Highberry 2017", 
    "Orange Blossom Jamboree 2017",
    "Purple Hatters Ball 2017",
    "The Werk Out 2017"
    
]
fest_genres.drop(skip, inplace=True)

In [241]:
fest_genres.head()

,abstract_beats,abstract_hip_hop,abstractro,acid_house,acid_jazz,acid_techno,acousmatic,acoustic_blues,acoustic_pop,adult_standards,african_rock,afrobeat,aggrotech,album_rock,albuquerque_indie,alt_indie_rock,alternative_americana,alternative_country,alternative_dance,alternative_emo,alternative_hip_hop,alternative_metal,alternative_pop,alternative_r&b,alternative_rock,alternative_roots_rock,ambient,anthem_emo,anthem_worship,anti_folk,appalachian_folk,arab_folk,arab_pop,argentine_indie,argentine_rock,art_rock,athens_indie,atmospheric_black_metal,aussietronica,austindie,australian_alternative_rock,australian_country,australian_dance,australian_hip_hop,australian_pop,avant_garde,avant_garde_jazz,avantgarde_metal,bachata,balearic,banda,bass_music,bass_trap,bay_area_indie,beatdown,bebop,belly_dance,big_band,big_beat,big_room,black_metal,black_sludge,blackgaze,bluegrass,blues,blues_rock,blues_rock_guitar,bmore,boogaloo,boogie_woogie,boston_rock,bounce,bow_pop,brass_band,brazilian_punk,breakbeat,breaks,brill_building_pop,british_blues,british_folk,british_invasion,britpop,broadway,brooklyn_indie,brostep,brutal_death_metal,bubblegum_pop,c86,c_pop,cabaret,cajun,canadian_country,canadian_indie,canadian_metal,canadian_pop,candy_pop,canterbury_scene,cantopop,catstep,ccm,cello,celtic,celtic_punk,celtic_rock,chamber_pop,chamber_psych,channel_pop,chanson,chaotic_hardcore,chicago_blues,chicago_house,chicago_indie,chicago_soul,children's_music,chill_lounge,chill_out_trance,chillstep,chillwave,chinese_indie_rock,christian_alternative_rock,christian_christmas,christian_dance,christian_hardcore,christian_metal,christian_music,christian_rock,christian_uplift,christmas,classic_funk_rock,classic_garage_rock,classic_rock,classic_russian_rock,classical,classify,colombian_rock,columbus_ohio_indie,comedy,comic,complextro,compositional_ambient,contemporary_country,contemporary_folk,contemporary_jazz,contemporary_post_bop,cool_jazz,corrosion,country,country_blues,country_christmas,country_dawn,country_gospel,country_road,country_rock,cowboy_western,crack_rock_steady,crossover_thrash,crunk,crust_punk,cumbia,dance_pop,dance_punk,dance_rock,dancehall,danish_pop,danish_pop_rock,dark_hardcore,dark_jazz,dark_wave,death_core,death_metal,deathgrind,deep_acoustic_pop,deep_australian_indie,deep_big_room,deep_canadian_indie,deep_chill,deep_christian_rock,deep_comedy,deep_cumbia_sonidera,deep_disco_house,deep_dub_techno,deep_euro_house,deep_funk,deep_funk_house,deep_german_indie,deep_groove_house,deep_house,deep_indie_r&b,deep_indie_singer_songwriter,deep_jazz_fusion,deep_latin_alternative,deep_melodic_euro_house,deep_melodic_metalcore,deep_minimal_techno,deep_new_americana,deep_pop_punk,deep_pop_r&b,deep_psychobilly,deep_psytrance,deep_regional_mexican,deep_soul_house,deep_southern_trap,deep_swedish_indie_pop,deep_taiwanese_pop,deep_talent_show,deep_tech_house,deep_texas_country,deep_trap,deep_tropical_house,deep_underground_hip_hop,deep_uplifting_trance,deep_vocal_jazz,delta_blues,desert_blues,destroy_techno,detroit_hip_hop,detroit_techno,didgeridoo,digital_hardcore,dirty_south_rap,disco,disco_house,dixieland,djent,doom_metal,doujin,downtempo,dream_pop,dreamo,drift,drill,drill_and_bass,drone,drone_folk,drone_metal,drum_and_bass,drumfunk,dub,dub_techno,dubstep,duranguense,dutch_house,dwn_trap,e6fi,east_coast_hip_hop,ebm,ectofolk,edm,electric_blues,electro,electro_bailando,electro_house,electro_industrial,electro_swing,electro_trash,electroclash,electrofox,electronic,electronic_trap,electronica,electropowerpop,emo,emo_punk,environmental,epicore,escape_room,ethereal_wave,etherpop,europop,exotica,experimental,experimental_psych,experimental_rock,fast_melodic_punk,fidget_house,filter_house,filthstep,fingerstyle,finnish_metal,flamenco,float_house,fluxwork,folk,folk_christmas,folk_metal,folk_pop,folk_punk,folk_rock,folkmusik,footwork,fourth_world,francoton,freak_folk,freakbeat,free_improvisation,free_jazz,freestyle,french_indie_pop,french_indietronica,french_pop,french_pu

In [244]:
fest_genres_norm = pd.DataFrame(
    preprocessing.normalize(fest_genres, norm='l1'), 
    columns=list(fest_genres.columns),
    index=list(fest_genres.index)
)

In [290]:
# pickel for later
with open('fest_genres_norm.pk1', 'wb') as picklefile:
    pickle.dump(fest_genres_norm, picklefile)

In [132]:
# load pickeled genre data
with open('fest_genres_norm.pk1', 'rb') as picklefile:
    fest_genres_norm = pickle.load(picklefile)

In [169]:
kmeans = KMeans(n_clusters=8, random_state=0)
kmeans.fit(fest_genres_norm)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [170]:
pd.Series(kmeans.labels_).value_counts()

0    37
3    19
1    17
5    16
6    14
7    13
4    13
2    10
dtype: int64

In [171]:
fest_genres['clusters'] = kmeans.labels_

In [192]:
single_cluster = fest_genres[fest_genres['clusters'] == 7]
festivals[festivals['name'].isin(list(single_cluster.index))]

,name,start_date,end_date,location,tickets,camping,website,description,lineup,url,poster,image,genres,unknown,num_bands,num_unknowns,unknown_percent
0,Savannah Music Festival 2017,2017-03-23,2017-04-08,"Savannah, GA",,No,http://www.savannahmusicfestival.org/,THE SAVANNAH MUSIC FESTIVAL IS DEDICATED TO PR...,"[the avett brothers, jason isbell, nikki lane,...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...,"{u'folk_pop': 6, u'indie_pop': 3, u'soul': 1, ...",[chicago blues meets gulf coast boogie: lurrie...,15,2,0.13
9,Rhythm N Blooms 2017,2017-04-07,2017-04-09,"Knoxville, TN",$65-$90,No,http://www.rhythmnbloomsfest.com/,Set amidst the urban artistic cityscape of the...,"[young the giant, gogol bordello, dave barnes,...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...,"{u'deep_funk': 1, u'viral_pop': 1, u'indietron...","[ruby amanfu & steelism, cruz contreras, midni...",35,4,0.11
21,High Water Festival 2017,2017-04-22,2017-04-23,"North Charleston, SC",$99.50-$274.50,No,http://www.highwaterfest.com/,"The husband and wife team behind the folk duo,...","[the avett brithers, the shins, shovels & rope...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...,"{u'deep_funk': 1, u'country_rock': 1, u'albuqu...","[the avett brithers, shrimp records family band]",22,2,0.09
26,Stagecoach Music Festival 2017,2017-04-28,2017-04-30,"Indio, CA",$299,Yes,http://www.stagecoachfestival.com/,,"[38 special, aaron lee tasjan, anderson east, ...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...,"{u'country_rock': 9, u'merseybeat': 1, u'folk_...",[the hillbenders present the who's tommy: a bl...,68,2,0.03
62,Nelsonville Music Festival 2017,2017-06-01,2017-06-04,"Nelsonville, OH",$140,Yes,http://www.nelsonvillefest.org/,,"[ween, cloud nothings, conor oberst, emmylou h...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...,"{u'country_rock': 3, u'indietronica': 2, u'fol...","[hellnaw, weedghost, aaron michael butler with...",33,4,0.12
77,LaureLive 2017,2017-06-10,2017-06-11,"Russel Township, OH",$50-$90,No,http://www.laurelive.com/,In addition to a weekend of great music in a b...,"[the head and the heart, gary clark jr., young...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...,"{u'christian_music': 1, u'indie_rockism': 3, u...",[liv cargile],31,1,0.03
85,Mountain Jam 2017,2017-06-16,2017-06-18,"Hunter Mountain, NY",$194,Yes,http://mountainjam.com/,,"[tom petty and the heartbreakers, steve miller...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...,"{u'austindie': 1, u'reggae_fusion': 1, u'deep_...",[the paul green rock academy],39,1,0.03
90,Clearwater Festival 2017,2017-06-17,2017-06-18,"Croton-On-Hudson, NY",,Yes,http://www.clearwaterfestival.org/,CLEARWATER'S GREAT HUDSON RIVER REVIVAL (AKA '...,"[lake street dive, cry cry cry reunion, arlo g...",https://www.musicfestivalwizard.com/festivals/...,[],https://www.musicfestivalwizard.com/wp-content...,"{u'ectofolk': 1, u'salsa': 1, u'boogaloo': 1, ...","[cry cry cry reunion, toshi reagon & big lovel...",36,10,0.28
96,Solid Sound Festival 2017,2017-06-23,2017-06-25,"North Adams, MA",$159.00,Yes,http://solidsoundfestival.com/,,"[alloy orchestra, andy shauf, the autumn defen...",https://www.musicfestivalwizard.com/festivals/...,[https://www.musicfestivalwizard.com/wp-conten...,https://www.musicfestivalwizard.com/wp-content...,"{u'protopunk': 1, u'art_rock': 1, u'deep_germa...","[alloy orchestra, dave alvin and phil alvin wi...",25,5,0.20
105,Winnipeg Fol

In [296]:
user_artists = ['crystal castles', 'ratatat', 'chvrches']

user = user_genres(user_artists)
user

,alternative_dance,brooklyn_indie,chillwave,dance_punk,digital_hardcore,electroclash,electronic,escape_room,indie_pop,indie_poptimism,indie_r&b,indie_rock,indietronica,metropopolis,new_rave,nintendocore,nu_gaze,pop,shimmer_pop,synthpop,tropical_house
User,0.10000,0.03333,0.03333,0.03333,0.03333,0.03333,0.03333,0.03333,0.10000,0.03333,0.03333,0.03333,0.10000,0.03333,0.06667,0.03333,0.03333,0.03333,0.03333,0.10000,0.03333


In [297]:
user_df = user.append(fest_genres_norm.ix[0,:])
user_df = user_df.fillna(0)
user_df

,alternative_dance,brooklyn_indie,chillwave,dance_punk,digital_hardcore,electroclash,electronic,escape_room,indie_pop,indie_poptimism,indie_r&b,indie_rock,indietronica,metropopolis,new_rave,nintendocore,nu_gaze,pop,shimmer_pop,synthpop,tropical_house,abstract_beats,abstract_hip_hop,abstractro,acid_house,acid_jazz,acid_techno,acousmatic,acoustic_blues,acoustic_pop,adult_standards,african_rock,afrobeat,aggrotech,album_rock,albuquerque_indie,alt_indie_rock,alternative_americana,alternative_country,alternative_emo,alternative_hip_hop,alternative_metal,alternative_pop,alternative_r&b,alternative_rock,alternative_roots_rock,ambient,anthem_emo,anthem_worship,anti_folk,appalachian_folk,arab_folk,arab_pop,argentine_indie,argentine_rock,art_rock,athens_indie,atmospheric_black_metal,aussietronica,austindie,australian_alternative_rock,australian_country,australian_dance,australian_hip_hop,australian_pop,avant_garde,avant_garde_jazz,avantgarde_metal,bachata,balearic,banda,bass_music,bass_trap,bay_area_indie,beatdown,bebop,belly_dance,big_band,big_beat,big_room,black_metal,black_sludge,blackgaze,bluegrass,blues,blues_rock,blues_rock_guitar,bmore,boogaloo,boogie_woogie,boston_rock,bounce,bow_pop,brass_band,brazilian_punk,breakbeat,breaks,brill_building_pop,british_blues,british_folk,british_invasion,britpop,broadway,brostep,brutal_death_metal,bubblegum_pop,c86,c_pop,cabaret,cajun,canadian_country,canadian_indie,canadian_metal,canadian_pop,candy_pop,canterbury_scene,cantopop,catstep,ccm,cello,celtic,celtic_punk,celtic_rock,chamber_pop,chamber_psych,channel_pop,chanson,chaotic_hardcore,chicago_blues,chicago_house,chicago_indie,chicago_soul,children's_music,chill_lounge,chill_out_trance,chillstep,chinese_indie_rock,christian_alternative_rock,christian_christmas,christian_dance,christian_hardcore,christian_metal,christian_music,christian_rock,christian_uplift,christmas,classic_funk_rock,classic_garage_rock,classic_rock,classic_russian_rock,classical,classify,colombian_rock,columbus_ohio_indie,comedy,comic,complextro,compositional_ambient,contemporary_country,contemporary_folk,contemporary_jazz,contemporary_post_bop,cool_jazz,corrosion,country,country_blues,country_christmas,country_dawn,country_gospel,country_road,country_rock,cowboy_western,crack_rock_steady,crossover_thrash,crunk,crust_punk,cumbia,dance_pop,dance_rock,dancehall,danish_pop,danish_pop_rock,dark_hardcore,dark_jazz,dark_wave,death_core,death_metal,deathgrind,deep_acoustic_pop,deep_australian_indie,deep_big_room,deep_canadian_indie,deep_chill,deep_christian_rock,deep_comedy,deep_cumbia_sonidera,deep_disco_house,deep_dub_techno,deep_euro_house,deep_funk,deep_funk_house,deep_german_indie,deep_groove_house,deep_house,deep_indie_r&b,deep_indie_singer_songwriter,deep_jazz_fusion,deep_latin_alternative,deep_melodic_euro_house,deep_melodic_metalcore,deep_minimal_techno,deep_new_americana,deep_pop_punk,deep_pop_r&b,deep_psychobilly,deep_psytrance,deep_regional_mexican,deep_soul_house,deep_southern_trap,deep_swedish_indie_pop,deep_taiwanese_pop,deep_talent_show,deep_tech_house,deep_texas_country,deep_trap,deep_tropical_house,deep_underground_hip_hop,deep_uplifting_trance,deep_vocal_jazz,delta_blues,desert_blues,destroy_techno,detroit_hip_hop,detroit_techno,didgeridoo,dirty_south_rap,disco,disco_house,dixieland,djent,doom_metal,doujin,downtempo,dream_pop,dreamo,drift,drill,drill_and_bass,drone,drone_folk,drone_metal,drum_and_bass,drumfunk,dub,dub_techno,dubstep,duranguense,dutch_house,dwn_trap,e6fi,east_coast_hip_hop,ebm,ectofolk,edm,electric_blues,electro,electro_bailando,electro_house,electro_industrial,electro_swing,electro_trash,electrofox,electronic_trap,electronica,electropowerpop,emo,emo_punk,environmental,epicore,ethereal_wave,etherpop,europop,exotica,experimental,experimental_psych,experimental_rock,fast_melodic_punk,fidget_house,filter_house,filthstep,fingerstyle,finnish_metal,flamenco,float_house,fluxwork,folk,folk_christmas,folk_metal,folk_pop,folk_punk,folk_rock,folkmusik,footwo

In [298]:
distances = pd.DataFrame(cosine_similarity(fest_genres_norm, user_df.ix[0,:]))

/home/aliphelan/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [299]:
distances.index = list(fest_genres.index)
distances.columns = ['distance']
distances.sort_values("distance")

,distance
Cavendish Beach Festival 2017,0.00000
Something Wonderful 2017,0.00000
Watershed Festival 2017,0.00000
Rockygrass Festival 2017,0.00000
Boots and Hearts 2017,0.00000
Dark Star Jubilee 2017,0.00000
Country Jam USA 2017,0.00000
Carolina Country Music Festival 2017,0.00000
Country Thunder Alberta 2017,0.00000
4 Peaks Music Festival 2017,0.00000


In [295]:
list(fest_genres_norm.columns)

[u'abstract_beats',
 u'abstract_hip_hop',
 u'abstractro',
 u'acid_house',
 u'acid_jazz',
 u'acid_techno',
 u'acousmatic',
 u'acoustic_blues',
 u'acoustic_pop',
 u'adult_standards',
 u'african_rock',
 u'afrobeat',
 u'aggrotech',
 u'album_rock',
 u'albuquerque_indie',
 u'alt_indie_rock',
 u'alternative_americana',
 u'alternative_country',
 u'alternative_dance',
 u'alternative_emo',
 u'alternative_hip_hop',
 u'alternative_metal',
 u'alternative_pop',
 u'alternative_r&b',
 u'alternative_rock',
 u'alternative_roots_rock',
 u'ambient',
 u'anthem_emo',
 u'anthem_worship',
 u'anti_folk',
 u'appalachian_folk',
 u'arab_folk',
 u'arab_pop',
 u'argentine_indie',
 u'argentine_rock',
 u'art_rock',
 u'athens_indie',
 u'atmospheric_black_metal',
 u'aussietronica',
 u'austindie',
 u'australian_alternative_rock',
 u'australian_country',
 u'australian_dance',
 u'australian_hip_hop',
 u'australian_pop',
 u'avant_garde',
 u'avant_garde_jazz',
 u'avantgarde_metal',
 u'bachata',
 u'balearic',
 u'banda',
 u'b

In [ ]:
distances = pd.DataFrame(cosine_similarity(fest_genres_norm, test_case_vector))

In [289]:
fest_genres_norm.drop('clusters', axis=1, inplace=True)

### Artist Recommender

In [34]:
with open('artist_average.pk1', 'rb') as picklefile:
    artist_average = pickle.load(picklefile)

In [39]:
artists = pd.DataFrame(artist_average).T
artists.head()

,acousticness,danceability,energy,genres,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
#familygrind,0.14,0.59,0.69,[],0.00,6.60,0.22,-8.78,0.40,84.85,0.57
'brink' brinkman,0.47,0.62,0.41,[],0.00,4.90,0.21,-9.55,0.03,110.47,0.60
070 shake,0.27,0.65,0.53,"[dwn trap, underground hip hop]",0.00,4.71,0.22,-8.56,0.09,114.37,0.30
10 string symphony,0.96,0.71,0.17,[deep new americana],0.02,3.60,0.10,-11.37,0.05,116.19,0.52
100 mile house,0.72,0.40,0.32,[],0.30,4.20,0.12,-13.54,0.03,120.91,0.33


In [42]:
df_genre = artists['genres'].str.get_dummies(sep=',')
df_genre.head()

,u'album rock',u'alternative country',u'alternative dance',u'alternative hip hop',u'alternative metal',u'alternative pop',u'alternative rock',u'alternative roots rock',u'ambient',u'anthem emo',u'anti-folk',u'argentine rock'],u'art rock',u'athens indie',u'aussietronica',u'austindie',u'australian alternative rock',u'australian dance',u'australian pop',u'avant-garde',u'avantgarde metal',u'balearic',u'bass music',u'bass trap',u'bay area indie',u'bebop',u'big beat',u'big room',u'black metal',u'blackgaze',u'bluegrass',u'blues',u'blues-rock',u'bmore',u'boogie-woogie',u'boston rock',u'brass band',u'breakbeat',u'brill building pop',u'british blues',u'british folk',u'british invasion',u'britpop',u'brooklyn indie',u'brostep',u'brutal death metal',u'bubblegum pop',u'c86',u'cabaret',u'canadian country',u'canadian indie',u'canadian metal',u'canadian pop',u'candy pop',u'canterbury scene',u'cantopop',u'catstep',u'ccm',u'celtic punk',u'celtic rock',u'celtic',u'chamber pop',u'chamber psych',u'chamber psych'],u'channel pop',u'chaotic hardcore',u'chicago blues',u'chicago house',u'chicago indie',u'chicago soul',u'chillstep',u'chillwave',u'christian alternative rock',u'christian christmas',u'christian hardcore',u'christian metal',u'christian music',u'christian rock',u'christian uplift',u'christmas',u'classic funk rock',u'classic garage rock',u'classic rock',u'classic russian rock',u'classify',u'comic',u'comic'],u'complextro',u'compositional ambient',u'contemporary country',u'contemporary jazz',u'contemporary post-bop',u'contemporary post-bop'],u'cool jazz',u'country blues',u'country christmas',u'country dawn',u'country gospel',u'country road',u'country road'],u'country rock',u'country',u'country'],u'cowboy western',u'crossover thrash',u'crunk',u'crust punk',u'cumbia',u'dance pop',u'dance rock',u'dance-punk',u'danish pop rock'],u'dark jazz',u'dark wave',u'death core',u'death metal',u'deathgrind',u'deep acoustic pop',u'deep australian indie',u'deep australian indie'],u'deep big room',u'deep big room'],u'deep canadian indie'],u'deep chill'],u'deep comedy'],u'deep euro house',u'deep funk',u'deep funk'],u'deep groove house',u'deep house',u'deep house'],u'deep indie r&b',u'deep melodic euro house',u'deep melodic euro house'],u'deep melodic metalcore',u'deep minimal techno',u'deep minimal techno'],u'deep new americana',u'deep pop punk'],u'deep pop r&b',u'deep regional mexican',u'deep soul house',u'deep southern trap',u'deep swedish indie pop',u'deep taiwanese pop',u'deep talent show',u'deep tech house',u'deep texas country',u'deep trap',u'deep tropical house',u'deep underground hip hop',u'delta blues',u'desert blues',u'destroy techno',u'detroit hip hop',u'detroit techno',u'digital hardcore',u'dirty south rap',u'disco house',u'disco',u'dixieland',u'djent',u'doo-wop',u'doom metal',u'downtempo',u'dream pop',u'dreamo',u'drift',u'drill and bass',u'drill',u'drone folk',u'drone metal',u'drone',u'drum and bass',u'drumfunk',u'dub techno',u'dub',u'dubstep',u'duranguense',u'dutch house',u'dwn trap',u'dwn trap'],u'e6fi',u'east coast hip hop',u'ebm',u'ectofolk',u'edm',u'electric blues',u'electro house',u'electro house'],u'electro swing',u'electro trash',u'electro',u'electro-industrial',u'electroclash',u'electrofox',u'electronic trap',u'electronic trap'],u'electronic',u'electronic'],u'electronica',u'emo punk',u'emo',u'escape room',u'escape room'],u'ethereal wave',u'etherpop',u'europop',u'experimental psych',u'experimental rock',u'experimental',u'fidget house',u'filter house',u'filter house'],u'filthstep',u'filthstep'],u'float house',u'fluxwork',u'folk christmas',u'folk metal',u'folk punk',u'folk rock',u'folk',u'folk-pop',u'folk-pop'],u'folkmusik',u'footwork',u'fourth world',u'freak folk',u'freakbeat',u'free improvisation',u'free jazz',u'french indie pop',u'french indietronica',u'french pop',u'french pop'],u'french reggae',u'french rock'],u'funk metal',u'funk rock',u'funk',u'funky tech house',u'future garage',u'future garage'],u'futurepop',u'g funk',u'gangster rap',u'ga

In [46]:
artist_recommend = pd.concat([artists, df_genre], axis=1)

In [48]:
artist_recommend.drop('genres', axis=1, inplace=True)
artist_recommend.head()

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence,u'album rock',u'alternative country',u'alternative dance',u'alternative hip hop',u'alternative metal',u'alternative pop',u'alternative rock',u'alternative roots rock',u'ambient',u'anthem emo',u'anti-folk',u'argentine rock'],u'art rock',u'athens indie',u'aussietronica',u'austindie',u'australian alternative rock',u'australian dance',u'australian pop',u'avant-garde',u'avantgarde metal',u'balearic',u'bass music',u'bass trap',u'bay area indie',u'bebop',u'big beat',u'big room',u'black metal',u'blackgaze',u'bluegrass',u'blues',u'blues-rock',u'bmore',u'boogie-woogie',u'boston rock',u'brass band',u'breakbeat',u'brill building pop',u'british blues',u'british folk',u'british invasion',u'britpop',u'brooklyn indie',u'brostep',u'brutal death metal',u'bubblegum pop',u'c86',u'cabaret',u'canadian country',u'canadian indie',u'canadian metal',u'canadian pop',u'candy pop',u'canterbury scene',u'cantopop',u'catstep',u'ccm',u'celtic punk',u'celtic rock',u'celtic',u'chamber pop',u'chamber psych',u'chamber psych'],u'channel pop',u'chaotic hardcore',u'chicago blues',u'chicago house',u'chicago indie',u'chicago soul',u'chillstep',u'chillwave',u'christian alternative rock',u'christian christmas',u'christian hardcore',u'christian metal',u'christian music',u'christian rock',u'christian uplift',u'christmas',u'classic funk rock',u'classic garage rock',u'classic rock',u'classic russian rock',u'classify',u'comic',u'comic'],u'complextro',u'compositional ambient',u'contemporary country',u'contemporary jazz',u'contemporary post-bop',u'contemporary post-bop'],u'cool jazz',u'country blues',u'country christmas',u'country dawn',u'country gospel',u'country road',u'country road'],u'country rock',u'country',u'country'],u'cowboy western',u'crossover thrash',u'crunk',u'crust punk',u'cumbia',u'dance pop',u'dance rock',u'dance-punk',u'danish pop rock'],u'dark jazz',u'dark wave',u'death core',u'death metal',u'deathgrind',u'deep acoustic pop',u'deep australian indie',u'deep australian indie'],u'deep big room',u'deep big room'],u'deep canadian indie'],u'deep chill'],u'deep comedy'],u'deep euro house',u'deep funk',u'deep funk'],u'deep groove house',u'deep house',u'deep house'],u'deep indie r&b',u'deep melodic euro house',u'deep melodic euro house'],u'deep melodic metalcore',u'deep minimal techno',u'deep minimal techno'],u'deep new americana',u'deep pop punk'],u'deep pop r&b',u'deep regional mexican',u'deep soul house',u'deep southern trap',u'deep swedish indie pop',u'deep taiwanese pop',u'deep talent show',u'deep tech house',u'deep texas country',u'deep trap',u'deep tropical house',u'deep underground hip hop',u'delta blues',u'desert blues',u'destroy techno',u'detroit hip hop',u'detroit techno',u'digital hardcore',u'dirty south rap',u'disco house',u'disco',u'dixieland',u'djent',u'doo-wop',u'doom metal',u'downtempo',u'dream pop',u'dreamo',u'drift',u'drill and bass',u'drill',u'drone folk',u'drone metal',u'drone',u'drum and bass',u'drumfunk',u'dub techno',u'dub',u'dubstep',u'duranguense',u'dutch house',u'dwn trap',u'dwn trap'],u'e6fi',u'east coast hip hop',u'ebm',u'ectofolk',u'edm',u'electric blues',u'electro house',u'electro house'],u'electro swing',u'electro trash',u'electro',u'electro-industrial',u'electroclash',u'electrofox',u'electronic trap',u'electronic trap'],u'electronic',u'electronic'],u'electronica',u'emo punk',u'emo',u'escape room',u'escape room'],u'ethereal wave',u'etherpop',u'europop',u'experimental psych',u'experimental rock',u'experimental',u'fidget house',u'filter house',u'filter house'],u'filthstep',u'filthstep'],u'float house',u'fluxwork',u'folk christmas',u'folk metal',u'folk punk',u'folk rock',u'folk',u'folk-pop',u'folk-pop'],u'folkmusik',u'footwork',u'fourth world',u'freak folk',u'freakbeat',u'free improvisation',u'free jazz',u'french indie pop',u'french indietronica',u'french pop',u'french pop'],u'french reggae',u'french rock'],u'funk metal',u'funk rock',u'funk',

In [51]:
artist_recommend.loc[festivals.ix[0,8]]

acousticness  \
the avett brothers                                                 0.48   
jason isbell                                                       0.47   
nikki lane                                                         0.14   
parker millsap                                                     0.40   
richard thompson                                                   0.42   
sarah jarosz                                                       0.79   
kolachi                                                            0.07   
hiss golden messenger                                              0.60   
travelin' mccourys                                                 0.48   
jeff austin band                                                   0.34   
chicago blues meets gulf coast boogie: lurrie b...                  NaN   
patrick watson                                                     0.73   
release the hounds: julian lage & chris eldridg...                  NaN   
bruce hornsby & the noisemakers                                    0.20   
the wood brothers                                                  0.48   

                                                           danceability  \
the avett brothers                                                 0.52   
jason isbell                                                       0.50   
nikki lane                                                         0.56   
parker millsap                                                     0.58   
richard thompson                                                   0.55   
sarah jarosz                                                       0.51   
kolachi                                                            0.43   
hiss golden messenger                                              0.65   
travelin' mccourys                                                 0.48   
jeff austin band                                                   0.56   
chicago blues meets gulf coast boogie: lurrie b...                  NaN   
patrick watson                                                     0.38   
release the hounds: julian lage & chris eldridg...                  NaN   
bruce hornsby & the noisemakers                                    0.52   
the wood brothers                                                  0.57   

                                                                 energy  \
the avett brothers                                                 0.46   
jason isbell                                                       0.28   
nikki lane                                                         0.71   
parker millsap                                                     0.36   
richard thompson                                                   0.44   
sarah jarosz                                                       0.27   
kolachi                                                            0.69   
hiss golden messenger                                              0.41   
travelin' mccourys                                                 0.66   
jeff austin band                                                   0.71   
chicago blues meets gulf coast boogie: lurrie b...                  NaN   
patrick watson                                                     0.35   
release the hounds: julian lage & chris eldridg...                  NaN   
bruce hornsby & the noisemakers                                    0.69   
the wood brothers                                                  0.44   

                                                       instrumentalness  \
the avett brothers                                                 0.00   
jason isbell                                                       0.00   
nikki lane                                                         0.00   
parker millsap                                                     0.00   
richard thompson                                                   0.00   
sarah 

In [52]:
festivals.ix[0,8]

[u'the avett brothers',
 u'jason isbell',
 u'nikki lane',
 u'parker millsap',
 u'richard thompson',
 u'sarah jarosz',
 u'kolachi',
 u'hiss golden messenger',
 u"travelin' mccourys",
 u'jeff austin band',
 u'chicago blues meets gulf coast boogie: lurrie bell, marcia ball, james cotton',
 u'patrick watson',
 u"release the hounds: julian lage & chris eldridge with aoife o'donovan",
 u'bruce hornsby & the noisemakers',
 u'the wood brothers']